In [13]:
import numpy as np
from pathlib import Path
import open3d as o3d
import json
from tqdm import tqdm

path = Path("./data/frames/")
cov = np.zeros(9).tolist()
colour = np.zeros((3, )).tolist()
pose = np.eye(4).reshape(16).tolist()

for i, file in tqdm(enumerate(list(sorted(path.glob("*.npy")))), total=len(tuple(path.glob("*.npy")))):
    points = np.load(file)
    
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(points)

    pc.estimate_normals()

    normals = np.asarray(pc.normals)

    data = []

    for point, normal in zip(points, normals):
        data.append({
            "position":point.tolist(),
            "normal":normal.tolist(),
            "colour":colour,
            "cov":cov,
            "valid":True
        })

    with open(file.parent / f"frame_{i:03}.json", "w") as f:
        json.dump({"pc":data, "pose":pose}, f)
    



100%|██████████| 36/36 [01:28<00:00,  2.46s/it]


In [14]:
from shutil import copyfile

dest = Path(r"/Users/chrisvail/Documents/CruxTest/matcher/examples/data")
source = Path(r"/Users/chrisvail/Documents/ReconstructionAnalysis/data/frames")

for frame in sorted(source.glob("*.json")):
    # print(frame.name)
    copyfile(frame, dest / frame.name)
    